---
layout: post
title:  "Ch03.Output Parsers"
date:   2025-06-27 02:00:00 +0700
categories: [LLM]
---

<script type="text/x-mathjax-config">
MathJax.Hub.Config({tex2jax: {inlineMath: [['$','$'], ['\\(','\\)']]}});
</script>
<script type="text/javascript" src="https://cdn.mathjax.org/mathjax/latest/MathJax.js?config=TeX-MML-AM_CHTML">
</script>

### 참조 사이트.
해당 Post는 <a href="https://wikidocs.net/book/14314">LangChain 위키독스</a>에 나와있는 예제와 흐름을 파악하는 용도 입니다.  

### Output Parser

Lang Chaing의 출력파서(Output Parser)는 LLM의 출력을 구조화된 형태로 변환하는 중요한 컴포넌트이다. 사용자가 원하는 Output Format으로 변환함으로 인하여, LLM간의 통신 혹은 후처리에서 이용이 가능하게 지정할 수 있다. 혹은 데이터를 파싱하여 사용할 수 있다.

**주요 특징**  
1. 다양성: LangChain은 많은 종류의 출력 파서를 제공합니다.
2. 스트리밍 지원: 많은 출력 파서들이 스트리밍을 지원합니다.
3. 확장성: 최소한의 모듈부터 복잡한 모듈까지 확장 가능한 인터페이스를 제공합니다.

**Output Example**  

기존 LLM Output
```code
**중요 내용 추출:**

1. **발신자:** 김철수 (chulsoo.kim@bikecorporation.me)
2. **수신자:** 이은채 (eunchae@teddyinternational.me)
3. **제목:** "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안
4. **요청 사항:**
   - ZENESIS 모델의 상세한 브로슈어 요청 (기술 사양, 배터리 성능, 디자인 정보 포함)
5. **미팅 제안:**
   - 날짜: 다음 주 화요일 (1월 15일)
   - 시간: 오전 10시
   - 장소: 귀사 사무실

6. **발신자 정보:**
   - 김철수, 상무이사, 바이크코퍼레이션
```

JSON 형식의 구조화 된 답변
```code
{
  "person": "김철수",
  "email": "chulsoo.kim@bikecorporation.me",
  "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
  "summary": "바이크코퍼레이션의 김철수 상무가 테디인터내셔널의 이은채 대리에게 신규 자전거 'ZENESIS' 모델에 대한 브로슈어 요청과 기술 사양, 배터리 성능, 디자인 정보 요청. 또한, 협력 논의를 위해 1월 15일 오전 10시에 미팅 제안.",
  "date": "1월 15일 오전 10시"
}
```


### Setting

#### Import Library

#### Appendix. Pydantic

LangChain, LangGraph에서 OutputParser로 많이 사용하는 Pydantic을 먼저 확인하자.  

Pydantic은 Python의 데이터 검증 및 설정 관리 라이브러리로, **Python type annotation**을 기반으로 데이터 유효성 검사, 자동 변환, 직렬화 등을 편리하게 수행할 수 있게 해줍니다. 

| 특징               | 설명                                                   |
| ---------------- | ---------------------------------------------------- |
| **타입 기반 유효성 검사** | Python 3의 타입 힌트를 사용해 자동으로 입력값 검증                     |
| **자동 타입 캐스팅**    | 가능한 경우 입력값을 지정한 타입으로 자동 변환                           |
| **데이터 직렬화/역직렬화** | `dict`, `json` 등으로 직렬화 가능                            |
| **중첩 모델 지원**     | 다른 Pydantic 모델을 필드로 가질 수 있음                          |
| **환경변수 설정 지원**   | `.env` 파일 등으로부터 설정 값을 불러올 수 있음 (Settings Management) |
| **속도**           | 내부적으로 Cython으로 최적화되어 있어 빠름                           |
| **엄격한 모드 지원**    | `StrictStr`, `StrictInt` 등을 사용해 자동 변환을 막을 수 있음       |


**기본 예제**

In [ ]:
from pydantic import validator
from pydantic import BaseModel, Field
from pydantic import StrictInt, StrictStr

class User(BaseModel):
    id: int
    name: str
    signup_ts: str = None
    friends: list[int] = []

user = User(id='123', name='Alice', friends=[1, '2', 3])
print(user)
print(user.dict())

id=123 name='Alice' signup_ts=None friends=[1, 2, 3]
{'id': 123, 'name': 'Alice', 'signup_ts': None, 'friends': [1, 2, 3]}


C:\Users\wjddy\AppData\Local\Temp\ipykernel_18652\193206898.py:12: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(user.dict())


**중첩 사용**

In [3]:
class Address(BaseModel):
    city: str
    zipcode: str

class Person(BaseModel):
    name: str
    age: int
    address: Address

data = {
    'name': 'John',
    'age': 30,
    'address': {
        'city': 'Seoul',
        'zipcode': '12345'
    }
}

p = Person(**data)
print(p.address.city)  # 'Seoul'

Seoul


**유효성 검사**

In [ ]:
class User(BaseModel):
    name: str
    age: int

    @validator('age') # age 필드의 유효성 검사
    def age_must_be_positive(cls, v):
        '''
        - cls: 클래스 자체. (클래스 메서드처럼 사용)
        - v: 해당 필드로 전달된 원본 값.
        '''
        if v <= 0:
            raise ValueError('Age must be positive')
        return v
    
user = User(name='Alice', age=30) # 정상 수행
user = User(name='Bob', age=-1) # validator에서 확인.

C:\Users\wjddy\AppData\Local\Temp\ipykernel_18652\3263515691.py:5: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator('age') # age 필드의 유효성 검사


ValidationError: 1 validation error for User
age
  Value error, Age must be positive [type=value_error, input_value=-1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

**변환 (alias & serialization)**

- <code>.dict()</code>: pydantic 결과를 dict 형태로 받음 - 내부 Python 변수명 기준
- <code>by_alias=True</code>: pydantic 결과를 dict 형태로 받음 - 외부 시스템에서 사용하는 alias 기준

In [ ]:
class User(BaseModel):
    user_id: int = Field(..., alias="userId")
    user_name: str = Field(..., alias="userName")

# 외부 JSON처럼 alias로 입력 받기
user = User(userId=1, userName="Alice")

# 내부 변수로 직접적으로 접근 불가능.
user = User(user_id=1, user_name="Alice")

ValidationError: 2 validation errors for User
userId
  Field required [type=missing, input_value={'user_id': 1, 'user_name': 'Alice'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
userName
  Field required [type=missing, input_value={'user_id': 1, 'user_name': 'Alice'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

In [ ]:
# Output은 모두 사용가능하나, key값이 다름.
print(user.dict())
print(user.dict(by_alias=True))

{'user_id': 1, 'user_name': 'Alice'}
{'userId': 1, 'userName': 'Alice'}


C:\Users\wjddy\AppData\Local\Temp\ipykernel_18652\3464781162.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(user.dict())
C:\Users\wjddy\AppData\Local\Temp\ipykernel_18652\3464781162.py:4: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(user.dict(by_alias=True))


**엄격 모드**

TypeHint로 지정하는 것이 아닌, data type을 강제로 지정한다.

In [ ]:
class Product(BaseModel):
    id: StrictInt
    name: StrictStr

product = Product(id=123, name='123') # 정상 수행
product = Product(id=123, name=123) # type error

ValidationError: 1 validation error for Product
name
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

그 외에, E-mail 주소 형식, 날짜 비교, 문자열 길이 등 다양한 변수의 제한 할 수 있다.

### Setting

#### Import Library

In [ ]:
import yaml
from typing import List
from pathlib import Path
from datetime import datetime

from langchain_openai import ChatOpenAI
from langchain.embeddings.base import Embeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import HumanMessage, AIMessage
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder, load_prompt
from langchain_core.output_parsers import StrOutputParser


from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser

In [18]:
from itertools import chain
from pydantic import BaseModel, Field

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

#### Model 선언

In [17]:
model = ChatOpenAI(
    base_url="http://localhost:8000/v1",
    api_key="EMPTY",
    model="Qwen/Qwen3-4B"
)

### PydanticOutputParser

**Argument**

```python
PydanticOutputParser(
    pydantic_object: Type[BaseModel],
    *,
    include_keys: Optional[List[str]] = None,
    exclude_keys: Optional[List[str]] = None,
    strict: bool = False,
    schema: Optional[Dict[str, Any]] = None
)
```

| 인자                | 타입                         | 설명                                                |
| ----------------- | -------------------------- | ------------------------------------------------- |
| `pydantic_object` | `Type[BaseModel]`          | 파싱 대상이 되는 Pydantic 모델 클래스. 필수 입력                  |
| `include_keys`    | `Optional[List[str]]`      | 출력 결과에서 포함할 키만 지정할 수 있음. 지정 시 해당 키만 포함됨           |
| `exclude_keys`    | `Optional[List[str]]`      | 출력 결과에서 제외할 키를 지정할 수 있음                           |
| `strict`          | `bool`                     | True로 설정 시, 모델의 스키마와 정확히 일치하지 않는 키가 있으면 오류 발생     |
| `schema`          | `Optional[Dict[str, Any]]` | 직접 schema를 지정해서 유효성 검사를 커스터마이징 가능 (일반적으로 사용되지 않음) |


**function**

- <code>get_format_instructions</code>: 언어 모델(LLM)이 **어떤 형식으로 출력을 생성해야 하는지** 알려주는 **지침 문자열(instruction)** 을 반환한다. 주로 프롬프트 템플릿에 포함되어, LLM이 출력 결과를 지정된 **Pydantic 모델 형식**에 맞게 만들도록 유도한다.
- <code>parse</code>: LLM이 출력한 **문자열(text)** 을 받아서, 이를 **Pydantic 모델 객체**로 파싱하는 메서드이다. 보통 LLM의 출력 결과를 Pydantic 기반으로 검증하거나, **구조화된 데이터**로 활용하고자 할 때 사용된다.

#### Input Dataset

In [19]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)

**W/O Output Parasers**

In [29]:
chain = prompt | model
answer = chain.invoke({"email_conversation": email_conversation})
print(answer.content.split("</think>\n\n")[1])

이메일에서 중요한 내용은 다음과 같습니다:

1. **발신자 정보**  
   - 이름: 김철수  
   - 직책: 바이크코퍼레이션 상무이사  
   - 이메일: chulsoo.kim@bikecorporation.me  

2. **수신자 정보**  
   - 이름: 이은채  
   - 직책: TEDDY INTERNATIONAL 대리  
   - 이메일: eunchae@teddyinternational.me  

3. **주제**  
   - "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안  

4. **핵심 요청**  
   - TEDDY INTERNATIONAL의 신규 자전거 **ZENESIS**에 대한 **상세 브로슈어** 요청  
     - 주요 정보: **기술 사양**, **배터리 성능**, **디자인 사항**  

5. **협력 논의를 위한 미팅 제안**  
   - 일정: **다음 주 화요일(1월 15일) 오전 10시**  
   - 장소: TEDDY INTERNATIONAL 사무실  
   - 목적: 유통 전략 및 마케팅 계획 논의  

6. **회사 소개**  
   - 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 **혁신과 품질**을 선도하는 기업으로, **장기적인 경험과 전문성**을 강조.  

이 정보는 협력 제안의 핵심 포인트를 간결하게 요약한 것입니다.


**W Output Parsers**

In [30]:
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")

# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


실제 결과를 살펴보게 되면, LLM에 어떤 지정을 하는 것이 아니라, Prompt에 강제로 내가 원하는 Format 출력을 도와주도록 추가하는 행위 이다.

In [34]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())

# chain 을 생성합니다.
chain = prompt | model

# chain 을 실행하고 결과를 출력합니다.
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

print(answer.content.split("</think>\n\n")[1])

이메일에서 중요한 내용은 다음과 같습니다:

1. **발신자 정보**  
   - 이름: 김철수  
   - 직책: 바이크코퍼레이션 상무이사  
   - 이메일: chulsoo.kim@bikecorporation.me  

2. **수신자 정보**  
   - 이름: 이은채  
   - 직책: TEDDY INTERNATIONAL 대리  
   - 이메일: eunchae@teddyinternational.me  

3. **주제**  
   - "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안  

4. **핵심 요청**  
   - TEDDY INTERNATIONAL의 신규 자전거 **ZENESIS**에 대한 **상세 브로슈어** 요청  
     - 주요 정보: **기술 사양**, **배터리 성능**, **디자인 사항**  

5. **협력 논의를 위한 미팅 제안**  
   - 일정: **다음 주 화요일(1월 15일) 오전 10시**  
   - 장소: TEDDY INTERNATIONAL 사무실  
   - 목적: 유통 전략 및 마케팅 계획 논의  

6. **회사 소개**  
   - 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 **혁신과 품질**을 선도하는 기업으로, **장기적인 경험과 전문성**을 강조.  

이 정보는 협력 제안의 핵심 포인트를 간결하게 요약한 것입니다.


위의 결과를 확인하게 되면, Prompt에 Format을 제한하여도 잘 작동하지 않는 걸 알 수 있다. LLM Model별로 Output은 다양할 것이며, 반드시 해당 Output으로 제한 할 수 없는 것을 알 수 있다.

<hr>
참조: <a href="https://wikidocs.net/book/14314">LangChain 위키독스</a><br>
참조: <a href="https://github.com/wjddyd66/LLM/tree/main">원본 코드</a><br>

코드에 문제가 있거나 궁금한 점이 있으면 wjddyd66@naver.com으로  Mail을 남겨주세요.